In [21]:
import time
import os
import csv
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm


In [22]:

def data_extractor(base_url, retries=3, delay=5):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }

    diagnosis_treatment_link = ""
    doctors_departments_link = ""

    for attempt in range(retries):
        try:
            response = requests.get(base_url, headers=headers, timeout=20)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract Diagnosis & Treatment link
            content1 = soup.find('a', id="et_genericNavigation_diagnosis-treatment")
            if not content1:
                # fallback: search by link text containing both words
                for a in soup.find_all('a'):
                    link_text = a.get_text(separator=' ').strip().lower()
                    if "diagnosis" in link_text and "treatment" in link_text:
                        content1 = a
                        break
            if content1:
                href1 = content1.get('href')
                diagnosis_treatment_link = f"https://www.mayoclinic.org{href1}" if href1 and href1.startswith("/") else href1

            # Extract Doctors & Departments link
            content2 = soup.find('a', id="et_genericNavigation_doctors-departments")
            if not content2:
                # fallback: search by link text containing both words
                for a in soup.find_all('a'):
                    link_text = a.get_text(separator=' ').strip().lower()
                    if "doctors" in link_text and "departments" in link_text:
                        content2 = a
                        break
            if content2:
                href2 = content2.get('href')
                doctors_departments_link = f"https://www.mayoclinic.org{href2}" if href2 and href2.startswith("/") else href2

            break  # success, exit retry loop

        except requests.exceptions.RequestException as e:
            print(f"[Attempt {attempt + 1}] Error fetching {base_url}: {e}")
            if attempt < retries - 1:
                time.sleep(delay)

    return diagnosis_treatment_link, doctors_departments_link

def web_scraping(base_url):
    # Define the expected headers in order
    expected_headers = ["disease", "main_link", "Diagnosis_treatment_link", "Doctors_departments_link"]
    
    # Check if file exists and read existing headers if it does
    file_exists = os.path.isfile("mayo_diseases.csv")
    existing_headers = []
    
    if file_exists:
        with open("mayo_diseases.csv", "r", encoding="utf-8") as file:
            reader = csv.reader(file)
            existing_headers = next(reader, [])
    
    # Determine if we need to write headers
    write_headers = not file_exists or existing_headers != expected_headers
    
    # Get the webpage content
    response = requests.get(base_url)
    if response.status_code != 200:
        print("Failed to retrieve page")
        exit()

    soup = BeautifulSoup(response.text, "html.parser")
    items = soup.select(".cmp-results-with-primary-name__see-link, .cmp-results-with-primary-name a")

    with open("mayo_diseases.csv", "a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        
        # Write headers if needed
        if write_headers:
            writer.writerow(expected_headers)
        
        for item in tqdm(items, desc="Scraping Diseases"):
            disease_name = item.text.strip()
            main_link = f"https://www.mayoclinic.org{item['href']}" if item['href'].startswith("/") else item['href']

            link1, link2 = data_extractor(main_link)
            
            # Create a row with all expected columns
            row_data = {
                "disease": disease_name,
                "main_link": main_link,
                "Diagnosis_treatment_link": link1,
                "Doctors_departments_link": link2
            }
            
            # If appending to existing file with different headers, align data with existing headers
            if file_exists and existing_headers:
                row = [row_data.get(header, "") for header in existing_headers]
            else:
                row = [row_data[header] for header in expected_headers]
            
            writer.writerow(row)

    print("Scraping Completed! Data Saved")

# Example usage:
# web_scraping("https://www.mayoclinic.org/diseases-conditions")

In [23]:
base_url = f"https://www.mayoclinic.org/diseases-conditions/index?letter=A"
web_scraping(base_url) 

Scraping Diseases: 100%|██████████| 132/132 [00:44<00:00,  2.98it/s]

Scraping Completed! Data Saved


In [20]:
from bs4 import BeautifulSoup


base_url="https://www.mayoclinic.org/diseases-conditions/egg-allergy/symptoms-causes/syc-20372115"
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
response = requests.get(base_url, headers=headers, timeout=20)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')
extraction_list = ["overview","symptoms","when-to-see-a-doctor","causes","complications","prevention","risk-factors"]

for i in extraction_list:
# Find the overview section by its aria-labelledby attribute
   overview_section = soup.find('section', {'aria-labelledby': i})

# Extract all paragraph text from the overview content
   overview_content = overview_section.find('div', class_='cmp-text__rich-content')
   overview_paragraphs = [p.get_text() for p in overview_content.find_all('p')]

# Join paragraphs into a single string
   overview_text = '\n\n'.join(overview_paragraphs)
   print(i)
   print(overview_text)

overview
Eggs are one of the most common allergy-causing foods for children.

Egg allergy symptoms usually occur a few minutes to a few hours after eating eggs or foods containing eggs. Signs and symptoms range from mild to severe and can include skin rashes, hives, nasal congestion, and vomiting or other digestive problems. Rarely, egg allergy can cause anaphylaxis — a life-threatening reaction.

Egg allergy can occur as early as infancy. Most children, but not all, outgrow their egg allergy before adolescence.
symptoms
Egg allergy reactions vary from person to person and usually occur soon after exposure to egg. Egg allergy symptoms can include:
when-to-see-a-doctor
See a doctor if you or your child has signs or symptoms of a food allergy shortly after eating eggs or an egg-containing product. If possible, see the doctor when the allergic reaction is occurring. This may help in making a diagnosis.

If you or your child has signs and symptoms of anaphylaxis, seek immediate emergency t

In [37]:
from bs4 import BeautifulSoup

def extract_overview(html_content):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    response = requests.get(base_url, headers=headers, timeout=20)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the Overview section - looking for h2 with text "Overview"
    overview_header = soup.find(lambda tag: tag.name == 'h2' and 'when-to-see-a-doctor"' in tag.text)
    
    if not overview_header:
        return "Overview section not found"
    
    # Initialize overview paragraphs
    overview_paragraphs = []
    
    # Get all siblings after the h2 until the next h2
    for sibling in overview_header.find_next_siblings():
        if sibling.name == 'h2':
            break  # Stop when we reach the next section
        if sibling.name == 'p':
            overview_paragraphs.append(sibling.get_text(strip=True))
    
    if not overview_paragraphs:
        return "No overview content found"
    
    return '\n\n'.join(overview_paragraphs)

# Example usage:
base_url="https://www.mayoclinic.org/diseases-conditions/hyperhidrosis/symptoms-causes/syc-20367152"

html_content = """ (your HTML content here) """
overview_text = extract_overview(base_url)
print(overview_text)

Overview section not found


In [43]:
from bs4 import BeautifulSoup
import requests

def extract_mayo_clinic_sections(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        main_content = soup.find('div', class_='content')
        if not main_content:
            return {"error": "Main content not found"}
        
        sections = {
            "Overview": None,
            "Symptoms": None,
            "When to see a doctor": None,
            "Causes": None,
            "Risk factors": None,
            "Complications": None,
            "Prevention": None
        }
        
        # Find all headings (h2 and h3)
        headings = main_content.find_all(['h2', 'h3'])
        
        for heading in headings:
            heading_text = heading.get_text(strip=True)
            
            # Handle the typo "dotor" in the HTML
            if "When to see a dotor" in heading_text:
                heading_text = "When to see a doctor"
                
            if heading_text in sections:
                content = []
                next_node = heading.next_sibling
                
                # Collect content until next heading
                while next_node and next_node.name not in ['h2', 'h3']:
                    if next_node.name == 'p':
                        content.append(next_node.get_text(strip=True))
                    elif next_node.name in ['ul', 'ol']:
                        items = [li.get_text(strip=True) for li in next_node.find_all('li')]
                        content.extend(items)
                    next_node = next_node.next_sibling
                
                sections[heading_text] = '\n'.join(content) if content else "No content found"
        
        return {k: v for k, v in sections.items() if v is not None}
    
    except requests.exceptions.RequestException as e:
        return {"error": f"Failed to fetch page: {str(e)}"}

# Example usage:
url = "https://www.mayoclinic.org/diseases-conditions/hyperhidrosis/symptoms-causes/syc-20367152"
sections = extract_mayo_clinic_sections(url)

for section_name, content in sections.items():
    print(f"=== {section_name.upper()} ===")
    print(content)
    print("\n" + "="*50 + "\n")

=== OVERVIEW ===
Hyperhidrosis (hi-pur-hi-DROE-sis) is excessive sweating that's not always related to heat or exercise. You may sweat so much that it soaks through your clothes or drips off your hands. Heavy sweating can disrupt your day and cause social anxiety and embarrassment.
Hyperhidrosis treatment usually helps. It often begins with antiperspirants. If these don't help, you may need to try different medications and therapies. In severe cases, your health care provider may suggest surgery to remove the sweat glands or to disconnect the nerves related to producing too much sweat.
Sometimes an underlying condition may be found and treated.


=== SYMPTOMS ===
The main symptom of hyperhidrosis is heavy sweating. This goes beyond the sweating from being in a hot environment, exercising, or feeling anxious or stressed. The type of hyperhidrosis that usually affects the hands, feet, underarms or face causes at least one episode a week when you're awake. And the sweating usually happens

In [8]:
import csv
from bs4 import BeautifulSoup
import requests
import os
from tqdm import tqdm

def extract_sections(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }

    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Try different possible main content containers
        main_content = soup.find('div', class_='content') or soup.find('article', id='main-content')
        if not main_content:
            return {}  # Don't return error; just skip

        # Section headings you want
        sections = {
            "Overview": None,
            "Symptoms": None,
            "When to see a doctor": None,
            "Causes": None,
            "Risk factors": None,
            "Complications": None,
            "Prevention": None
        }

        # Find all headings inside main content
        headings = main_content.find_all(['h2', 'h3'])
        for idx, heading in enumerate(headings):
            heading_text = heading.get_text(strip=True)
            # Fix potential typo in heading
            if "When to see a dotor" in heading_text:
                heading_text = "When to see a doctor"
            # If it's a section we care about
            if heading_text in sections:
                # Gather all paragraph and list content until next heading
                content = []
                next_node = heading.find_next_sibling()
                while next_node and next_node.name not in ['h2', 'h3']:
                    if next_node.name == 'p':
                        content.append(next_node.get_text(strip=True))
                    elif next_node.name in ['ul', 'ol']:
                        items = [li.get_text(strip=True) for li in next_node.find_all('li')]
                        content.extend(items)
                    next_node = next_node.find_next_sibling()
                if content:
                    sections[heading_text] = '\n'.join(content)
        # Only return sections with content
        return {k: v for k, v in sections.items() if v}
    except Exception as e:
        # Just return empty dict for error
        return {}

def update_csv_with_sections(csv_file):
    # Read existing data and headers
    rows = []
    existing_headers = []
    if os.path.exists(csv_file):
        with open(csv_file, 'r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            existing_headers = reader.fieldnames
            rows = list(reader)

    # List of desired section headers
    section_headers = [
        'Overview', 'Symptoms', 'When to see a doctor',
        'Causes', 'Risk factors', 'Complications', 'Prevention'
    ]
    # Columns from csv plus any missing section headers
    all_headers = existing_headers.copy() if existing_headers else ['disease', 'main_link']
    for header in section_headers:
        if header not in all_headers:
            all_headers.append(header)

    # Process each row and update with extracted sections
    updated_rows = []
    for row in tqdm(rows):
        # Only process if main_link exists
        if 'main_link' in row and row['main_link']:
            print(f"Processing: {row.get('disease', 'Unknown')}")
            sections = extract_sections(row['main_link'])
            # Update row with whatever we could extract
            for section, content in sections.items():
                row[section] = content
        updated_rows.append(row)

    # Write back to CSV with updated headers and rows
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=all_headers)
        writer.writeheader()
        writer.writerows(updated_rows)
    print(f"CSV file updated successfully with {len(updated_rows)} rows")

# Example usage
if __name__ == "__main__":
    csv_file = "mayo_diseases.csv"
    update_csv_with_sections(csv_file)

  0%|          | 0/132 [00:00<?, ?it/s]

Processing: Atrial fibrillation


  1%|          | 1/132 [00:00<00:48,  2.70it/s]

Processing: Hyperhidrosis


  2%|▏         | 2/132 [00:00<00:49,  2.62it/s]

...

Processing: Avascular necrosis (osteonecrosis)


 99%|█████████▉| 131/132 [00:45<00:00,  2.85it/s]

Processing: Atrioventricular nodal reentry tachycardia (AVNRT)


100%|██████████| 132/132 [00:45<00:00,  2.90it/s]

CSV file updated successfully with 132 rows


In [24]:
import csv
from bs4 import BeautifulSoup
import requests
import os
from tqdm import tqdm

# Section slugs for aria-labelledby (new design) and pretty names
SECTION_SLUGS = [
    ("overview", "Overview"),
    ("symptoms", "Symptoms"),
    ("when-to-see-a-doctor", "When to see a doctor"),
    ("causes", "Causes"),
    ("risk-factors", "Risk factors"),
    ("complications", "Complications"),
    ("prevention", "Prevention"),
]

def extract_sections(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }

    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        result_sections = {}

        # Try method 1: Old layout (div.content or article#main-content)
        main_content = soup.find('div', class_='content') or soup.find('article', id='main-content')
        if main_content:
            headings = main_content.find_all(['h2', 'h3'])
            for heading in headings:
                heading_text = heading.get_text(strip=True)
                # Fix typo
                if "When to see a dotor" in heading_text:
                    heading_text = "When to see a doctor"
                # If it's a section we care about
                for _, section_name in SECTION_SLUGS:
                    if heading_text == section_name:
                        content = []
                        next_node = heading.find_next_sibling()
                        while next_node and next_node.name not in ['h2', 'h3']:
                            if next_node.name == 'p':
                                content.append(next_node.get_text(strip=True))
                            elif next_node.name in ['ul', 'ol']:
                                items = [li.get_text(strip=True) for li in next_node.find_all('li')]
                                content.extend(items)
                            next_node = next_node.find_next_sibling()
                        if content:
                            result_sections[section_name] = '\n'.join(content)
        # Try method 2: New layout (section[aria-labelledby] + cmp-text__rich-content)
        for slug, section_name in SECTION_SLUGS:
            if section_name in result_sections:
                continue  # Already found by old method
            # Find aria-labelledby section
            section = soup.find('section', {'aria-labelledby': slug})
            if section:
                content_div = section.find('div', class_='cmp-text__rich-content')
                if content_div:
                    paragraphs = [p.get_text(strip=True) for p in content_div.find_all('p')]
                    if paragraphs:
                        result_sections[section_name] = '\n\n'.join(paragraphs)
        return result_sections
    except Exception as e:
        # Skip on error, return empty
        return {}

def update_csv_with_sections(csv_file):
    # Read existing data and headers
    rows = []
    existing_headers = []
    if os.path.exists(csv_file):
        with open(csv_file, 'r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            existing_headers = reader.fieldnames
            rows = list(reader)

    section_headers = [section for slug, section in SECTION_SLUGS]
    all_headers = existing_headers.copy() if existing_headers else ['disease', 'main_link']
    for header in section_headers:
        if header not in all_headers:
            all_headers.append(header)

    # --- SINGLE tqdm bar for ALL rows ---
    updated_rows = []
    with tqdm(total=len(rows), desc="Processing diseases", unit="disease") as pbar:
        for row in rows:
            if 'main_link' in row and row['main_link']:
                # (Optional) tqdm.write() to log messages without disrupting the bar
                tqdm.write(f"Processing: {row.get('disease', 'Unknown')}")
                sections = extract_sections(row['main_link'])
                for section, content in sections.items():
                    row[section] = content
            updated_rows.append(row)
            pbar.update(1)  # Always update once per disease

    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=all_headers)
        writer.writeheader()
        writer.writerows(updated_rows)
    print(f"CSV file updated successfully with {len(updated_rows)} rows")

# Example usage
if __name__ == "__main__":
    csv_file = "mayo_diseases.csv"
    update_csv_with_sections(csv_file)

Processing diseases:   0%|          | 0/132 [00:00<?, ?disease/s]

Processing: Atrial fibrillation


Processing diseases:   1%|          | 1/132 [00:00<00:42,  3.09disease/s]

Processing: Hyperhidrosis


Processing diseases:   2%|▏         | 2/132 [00:00<00:45,  2.83disease/s]

...

Processing: Avascular necrosis (osteonecrosis)


Processing diseases:  99%|█████████▉| 131/132 [00:46<00:00,  3.01disease/s]

Processing: Atrioventricular nodal reentry tachycardia (AVNRT)


Processing diseases: 100%|██████████| 132/132 [00:46<00:00,  2.82disease/s]

CSV file updated successfully with 132 rows


In [ ]:
import csv
from bs4 import BeautifulSoup
import requests
import os
from tqdm import tqdm

# Section slugs for aria-labelledby (new design) and pretty names
SECTION_SLUGS = [
    ("diagnosis", "Diagnosis"),
    ("treatment", "Treatment"),
    ("coping-and-support", "Coping and support"),
    ("preparing-for-your-appointment", "Preparing for your appointment"),
    ("lifestyle-and-home-remedies", "Lifestyle and home remedies")
]

def extract_sections(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }

    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        result_sections = {}

        # Try method 1: Old layout (div.content or article#main-content)
        main_content = soup.find('div', class_='content') or soup.find('article', id='main-content')
        if main_content:
            headings = main_content.find_all(['h2', 'h3'])
            for heading in headings:
                heading_text = heading.get_text(strip=True)
                # Fix typo
                if "When to see a dotor" in heading_text:
                    heading_text = "When to see a doctor"
                # If it's a section we care about
                for _, section_name in SECTION_SLUGS:
                    if heading_text == section_name:
                        content = []
                        next_node = heading.find_next_sibling()
                        while next_node and next_node.name not in ['h2', 'h3']:
                            if next_node.name == 'p':
                                content.append(next_node.get_text(strip=True))
                            elif next_node.name in ['ul', 'ol']:
                                items = [li.get_text(strip=True) for li in next_node.find_all('li')]
                                content.extend(items)
                            next_node = next_node.find_next_sibling()
                        if content:
                            result_sections[section_name] = '\n'.join(content)
        # Try method 2: New layout (section[aria-labelledby] + cmp-text__rich-content)
        for slug, section_name in SECTION_SLUGS:
            if section_name in result_sections:
                continue  # Already found by old method
            # Find aria-labelledby section
            section = soup.find('section', {'aria-labelledby': slug})
            if section:
                content_div = section.find('div', class_='cmp-text__rich-content')
                if content_div:
                    paragraphs = [p.get_text(strip=True) for p in content_div.find_all('p')]
                    if paragraphs:
                        result_sections[section_name] = '\n\n'.join(paragraphs)
        return result_sections
    except Exception as e:
        # Skip on error, return empty
        return {}

def update_csv_with_sections(csv_file):
    # Read existing data and headers
    rows = []
    existing_headers = []
    if os.path.exists(csv_file):
        with open(csv_file, 'r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            existing_headers = reader.fieldnames
            rows = list(reader)

    section_headers = [section for slug, section in SECTION_SLUGS]
    all_headers = existing_headers.copy() if existing_headers else ['disease', 'main_link']
    for header in section_headers:
        if header not in all_headers:
            all_headers.append(header)

    # --- SINGLE tqdm bar for ALL rows ---
    updated_rows = []
    with tqdm(total=len(rows), desc="Processing diseases", unit="disease") as pbar:
        for row in rows:
            if 'main_link' in row and row['Diagnosis_treatment_link']:
                # (Optional) tqdm.write() to log messages without disrupting the bar
                tqdm.write(f"Processing: {row.get('disease', 'Unknown')}")
                sections = extract_sections(row['Diagnosis_treatment_link'])
                for section, content in sections.items():
                    row[section] = content
            updated_rows.append(row)
            pbar.update(1)  # Always update once per disease

    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=all_headers)
        writer.writeheader()
        writer.writerows(updated_rows)
    print(f"CSV file updated successfully with {len(updated_rows)} rows")

# Example usage
if __name__ == "__main__":
    csv_file = "mayo_diseases.csv"
    update_csv_with_sections(csv_file)

Processing diseases:   0%|          | 0/132 [00:00<?, ?disease/s]

Processing: Atrial fibrillation


Processing diseases:   1%|          | 1/132 [00:00<00:54,  2.39disease/s]

Processing: Hyperhidrosis


Processing diseases:   2%|▏         | 2/132 [00:01<01:08,  1.90disease/s]

...

Processing: Avascular necrosis (osteonecrosis)


Processing diseases:  99%|█████████▉| 131/132 [01:31<00:00,  1.92disease/s]

Processing: Atrioventricular nodal reentry tachycardia (AVNRT)


Processing diseases: 100%|██████████| 132/132 [01:34<00:00,  1.39disease/s]

CSV file updated successfully with 132 rows


In [31]:
extraction_list = ["overview","symptoms","when-to-see-a-doctor","causes","complications","prevention","risk-factors"]

In [40]:
import pandas as pd

In [41]:
df = pd.read_csv("mayo_diseases.csv")

In [42]:
df.isnull().sum()

disease                            0
main_link                          0
Diagnosis_treatment_link           1
Doctors_departments_link          25
Diagnosis                          1
Treatment                          1
Coping and support                76
Preparing for your appointment     6
dtype: int64

In [44]:
df.head()

,disease,main_link,Diagnosis_treatment_link,Doctors_departments_link,Diagnosis,Treatment,Coping and support,Preparing for your appointment
0,Atrial fibrillation,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,You may not know you have atrial fibrillation ...,The goals of atrial fibrillation treatment are...,NaN,If you have an irregular or pounding heartbeat...
1,Hyperhidrosis,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,Diagnosing hyperhidrosis may start with your h...,Treating hyperhidrosis may start with treating...,Hyperhidrosis can be the cause of discomfort a...,You may start by seeing your primary care prov...
2,Bartholin's cyst,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,"To diagnose a Bartholin's cyst, your doctor ma...",Often a Bartholin's cyst requires no treatment...,NaN,Your first appointment will likely be with eit...
3,Infant reflux,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,NaN,"To diagnose infant reflux, a healthcare profes...","For most babies, making some changes to feedin...",NaN,You may start by seeing your baby's primary he...
4,Hidradenitis suppurativa,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,Hidradenitis suppurativa can be mistaken for p...,"Treatment with medicines, surgery or both can ...",Hidradenitis suppurativa can be a challenge to...,You'll likely first see your primary care prov...


In [46]:
import pandas as pd

# Load your CSV
df = pd.read_csv('mayo_diseases.csv')

# List of columns to check for all-null per disease
cols_to_check = [
'Preparing for your appointment'
]

# Find rows where all these columns are null (or empty string, or only whitespace)
mask = df[cols_to_check].isnull() | (df[cols_to_check].applymap(lambda x: str(x).strip() == ""))

# Disease rows where ALL of the specified columns are null/empty
fully_null = df[mask.all(axis=1)]

print(f"Diseases with all info columns missing ({len(fully_null)} found):")


Diseases with all info columns missing (6 found):


C:\Users\harme\AppData\Local\Temp\ipykernel_23608\687039859.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df[cols_to_check].isnull() | (df[cols_to_check].applymap(lambda x: str(x).strip() == ""))


In [47]:
fully_null.head()

,disease,main_link,Diagnosis_treatment_link,Doctors_departments_link,Diagnosis,Treatment,Coping and support,Preparing for your appointment
14,Radiation sickness,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,NaN,When a person has experienced known or probabl...,The treatment goals for radiation sickness are...,NaN,NaN
15,Radiation sickness,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,NaN,When a person has experienced known or probabl...,The treatment goals for radiation sickness are...,NaN,NaN
17,ARDS,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,There's no specific test for ARDS. Healthcare ...,The first goal in treating ARDS is to improve ...,"Recovery from ARDS can take time, and you're l...",NaN
69,Aortic aneurysm,https://www.mayoclinic.org/diseases-conditions...,NaN,https://www.mayoclinic.org/diseases-conditions...,NaN,NaN,NaN,NaN
71,Popliteal artery aneurysm,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,https://www.mayoclinic.org/diseases-conditions...,"To diagnose popliteal artery aneurysm, a healt...",Treatment of popliteal artery aneurysm depends...,NaN,NaN
